<a href="https://colab.research.google.com/github/VivianLexy/Ecco-test-master/blob/master/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch


In [ ]:
# Install required packages
!pip install transformers datasets accelerate bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import json

# Mount Google Drive (if necessary)
from google.colab import drive
drive.mount('/content/drive')

# Function to load your corpus
def load_corpus(filepath):
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Check if data is a list
        if isinstance(data, list):
            return Dataset.from_list(data)  # Use from_list since data is already a list
        else:
            print("Error: Loaded data is not a list")
            return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Load your dataset
file_path = "/content/drive/MyDrive/corpus_tokenized.json"  # Update with your file path
dataset = load_corpus(file_path)

# Check if the dataset is loaded correctly
if dataset is None:
    print("Dataset loading failed.")
else:
    print("Dataset loaded successfully.")
    print(f"Dataset size: {len(dataset)}")
    print(f"Dataset columns: {dataset.column_names}")

# Load pre-trained model and tokenizer
model_name = "gpt2"  # You can change this to other models like "gpt2-medium", "gpt2-large", etc.
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# If your tokenized dataset does not have a pad_token set, you can define it
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token

# Check if the required keys exist in your dataset
if not all(key in dataset.column_names for key in ["input_ids", "attention_mask"]):
    print("Error: The dataset must include 'input_ids' and 'attention_mask' columns.")

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,  # Adjust based on your GPU memory
    per_device_eval_batch_size=2,
    num_train_epochs=3,  # Adjust as needed
    logging_dir="./logs",
    learning_rate=2e-5,
    weight_decay=0.01,
    remove_unused_columns=False,  # Prevent column removal errors
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,  # Use your preprocessed dataset directly
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset loaded successfully.
Dataset size: 24
Dataset columns: ['metadata', 'sections']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Error: The dataset must include 'input_ids' and 'attention_mask' columns.


RuntimeError: Could not infer dtype of dict